In [1]:
from dotenv import load_dotenv
import os

load_dotenv('db.env', override=True)
NEO4J_URI = os.getenv('NEO4J_URI')
NEO4J_USERNAME = os.getenv('NEO4J_USERNAME')
NEO4J_PASSWORD = os.getenv('NEO4J_PASSWORD')

In [2]:
import torch

In [3]:
pcst_output = torch.load('stark_qa_v17/processed/test_data_nodes.pt')

/var/folders/rp/zlbqb_1d7j53cg1t8g_qq8q40000gn/T/ipykernel_83971/4185301223.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  pcst_output = torch.load('stark_qa_v17/proces

In [4]:
pcst_output.keys()

dict_keys(['correct_nodes', 'topk_nodes', 'subgraph1_nodes', 'pcst_nodes'])

In [5]:
from stark_qa import load_qa

In [6]:
qa_dataset = load_qa("prime")
qa_raw_test = qa_dataset.get_subset('test')

Use file from /Users/sbr/.cache/huggingface/hub/datasets--snap-stanford--stark/snapshots/88269e23e90587f99476c5dd74e235a0877e69be/qa/prime/stark_qa/stark_qa_human_generated_eval.csv.


In [7]:
query_embedding_dict = torch.load('data-loading/emb/prime/text-embedding-ada-002/query/query_emb_dict.pt')

In [8]:
ordered_pcst_nodes_20 = {}

In [9]:
from neo4j import Driver, GraphDatabase

In [10]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [11]:
with GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD)) as driver:
    # loop over key values of pcst_output['pcst_nodes'] dict
    for query_id, pcst_nodes in pcst_output['pcst_nodes'].items():
        # get the query text
        query_emb = query_embedding_dict[query_id]

        res = driver.execute_query("""
        UNWIND $nodeIds AS nodeId
        MATCH (node:_Entity_ {nodeId:nodeId}) RETURN node.nodeId as nodeId, node.textEmbedding AS textEmbedding
        """,
                                   parameters_={
                                       "nodeIds": pcst_nodes})
        node_embs = pd.DataFrame([rec.data() for rec in res.records])
        embeddings = np.vstack(node_embs['textEmbedding'].values)
        cos_sim = cosine_similarity(embeddings, query_emb.reshape(1,-1)).ravel()
        top_n_indices = np.argsort(cos_sim)[-20:][::-1]
        top_n_nodeIds = node_embs.iloc[top_n_indices]['nodeId'].to_numpy()

        breakpoint()
        ordered_pcst_nodes_20[query_id] = top_n_nodeIds

In [12]:
len(ordered_pcst_nodes_20)

2801

In [15]:
ordered_pcst_nodes_20[10630]

array([ 17445,  17466,  17457,  14092,  18369,  14090,  18807,  18024,
        17974,  14101,  18027,  18961,  19715,  16926,  18355,  14091,
       115411,  17823,  16154,  18025])

In [16]:
from compute_metrics import compute_intermediate_metrics

In [17]:
pcst_output.keys()

dict_keys(['correct_nodes', 'topk_nodes', 'subgraph1_nodes', 'pcst_nodes'])

In [18]:
len(pcst_output['correct_nodes'])

2801

In [19]:
compute_intermediate_metrics(pcst_output['correct_nodes'], ordered_pcst_nodes_20)

F1:              0.059965080657517315
Precision:       0.03570153516601214
Recall:          0.35934884544441004
Exact hit@1:     0.12602641913602286
Exact hit@5:     0.3156015708675473
Exact hit@any:   0.4423420207068904
Recall@20:       0.35934884544441004
MRR:             0.20963870057779188
Num predictions: 20.0


In [20]:
eval_outs = torch.load('stark_qa_v17/models/v17_gnn-llm-llama3.1-8b_eval_outs.pt')

In [21]:
evals = pd.concat([pd.DataFrame(d) for d in eval_outs])

In [22]:
evals

,pred,question,desc,label
0,integral component of plasma membrane|plasma m...,Question: Which cell structures are involved i...,"node_id,vector_similarity,node_attr\n17434,0.8...",cytoplasm|integral component of plasma membran...
1,tcap[/s][/s]pdlim3[/s][/s]tmem151b[/s][/s]lcp2[/s,Question: Which genes or proteins are expresse...,"node_id,vector_similarity,node_attr\n64835,0.8...",adh1b
2,thiamine[/s][/s]thiamine|vitamin b1[/s][/s][/s...,Question: Could you suggest any medications ef...,"node_id,vector_similarity,node_attr\n14982,0.8...",nicotinamide
3,pi3kr1|pi3kr2[/s][/s]pi3kr1[/s]pi3kr2[/s]pi3kr,Question: Which gene or protein responsible fo...,"node_id,vector_similarity,node_attr\n43,0.8684...",pik3r1|pik3r2
0,familial supernumerary nipples[/s][/s]accessor...,Question: Which autosomal dominant diseases ar...,"node_id,vector_similarity,node_attr\n38395,0.8...",familial supernumerary nipples
...,...,...,...,...
0,defective slc4a4 causes anion exchanger defici...,Question: Please find a pathway involved in SL...,"node_id,vector_similarity,node_attr\n62710,0.8...",defective slc4a4 causes renal tubular acidosis...
1,verapamil|diltiazem[/s][/s][/p]verapamil[/s]di...,"Question: Which medications, when co-administe...","node_id,vector_similarity,node_attr\n14271,0.8...",isradipine|nisoldipine|verapamil|lercanidipine...
2,kcnk5|kcnk4|kcnk18|kcnk6|kcnk16|kcnk2|kcn,Question: Please find genes that code for prot...,"node_id,vector_similarity,node_attr\n59703,0.8...",kcnq4|kcnf1|kcnk5|kcnk7|kcnk17|kcnk1|kcnc4|kcn...
3,sting mediated induction of host immune respon...,Question: Could you find out which cellular si...,"node_id,vector_similarity,node_attr\n107494,0....",regulation of innate immune responses to cytos...


In [23]:
len(qa_raw_test.indices)

2801

In [24]:
i = 0
node_pred_from_text = {}
for pred, label in zip(evals.pred.tolist(), evals.label.tolist()):
    pred = pred.split('[/s]')[0].strip().split('|')
    # covert every string to all uppercase
    pred = [p.upper() for p in pred]
    with GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD)) as driver:
        res = driver.execute_query("""
        MATCH (n) WHERE toUpper(n.name) IN $pred RETURN n.nodeId as nodeId
        """,
                                   parameters_={
                                       "pred": pred})
        pred_ids = [rec.data() for rec in res.records]
        node_ids = [item['nodeId'] for item in pred_ids]
        node_pred_from_text[i] = node_ids
        i += 1

In [24]:
node_pred_from_text

{0: [55842, 56073, 56207, 56263, 56436, 126196],
 1: [2773],
 2: [14130],
 3: [],
 4: [30638],
 5: [47408],
 6: [2582],
 7: [40924, 42036, 103931],
 8: [16181, 16635],
 9: [14019, 14023, 14024, 14028, 14320, 15530],
 10: [111],
 11: [31535],
 12: [38162],
 13: [41320, 48187, 50011],
 14: [84276, 94419],
 15: [28176],
 16: [32617],
 17: [7528],
 18: [28455, 24225],
 19: [64512, 64876, 66747],
 20: [3650, 4807],
 21: [37440],
 22: [37102],
 23: [86116],
 24: [53213, 102604, 111174],
 25: [14945, 15205, 15230],
 26: [33654, 25608],
 27: [111],
 28: [14051],
 29: [39750],
 30: [12390],
 31: [32724, 32731],
 32: [15142],
 33: [14794, 15030, 15187, 15572],
 34: [28896],
 35: [27641],
 36: [39682],
 37: [9967],
 38: [64778, 64876, 67302],
 39: [19622],
 40: [64512, 64799, 66747],
 41: [1272, 7402, 56557],
 42: [36187],
 43: [5843],
 44: [28242],
 45: [88259],
 46: [56241, 56436, 126196],
 47: [55992, 56150, 56174, 124245],
 48: [62401],
 49: [5385, 5481, 11624, 11881, 12713],
 50: [83828],
 5

In [33]:
consecutive_correct_nodes = {i:v for i,v in enumerate(pcst_output['correct_nodes'].values())}

In [34]:
consecutive_correct_nodes

{0: [56241, 55842, 56174, 56263],
 1: [10833],
 2: [15179],
 3: [43, 2140],
 4: [30638],
 5: [62592],
 6: [59232],
 7: [52067, 53456, 46931, 40628, 103931],
 8: [18223],
 9: [14330, 14505, 14508, 14512, 14522, 14141],
 10: [111],
 11: [99105, 99106, 99107, 33271, 99109, 30376, 31629, 99247, 94839, 27609],
 12: [31754, 27547, 38252],
 13: [45193, 47627, 50011, 103928, 112264],
 14: [94418, 94210, 94316],
 15: [27831],
 16: [29113],
 17: [8034,
  57603,
  61258,
  61259,
  61260,
  61261,
  8878,
  61262,
  61263,
  61264,
  61265,
  61266,
  61267,
  61268,
  61269,
  5874,
  57658,
  701],
 18: [24225, 23564, 22757, 22759],
 19: [63952, 64778, 63235, 64876],
 20: [13664, 11205, 6887, 5448, 3145, 3147, 11641, 5884, 765],
 21: [37440],
 22: [96249, 96941],
 23: [23162, 23915],
 24: [108356, 44625, 106193, 103667, 46101, 42615, 111482, 48219, 112316],
 25: [16019],
 26: [94307, 22349, 22335],
 27: [111],
 28: [14558],
 29: [39750],
 30: [12390],
 31: [32731],
 32: [14844],
 33: [15572],
 

In [35]:
compute_intermediate_metrics(consecutive_correct_nodes, node_pred_from_text)

F1:              0.2697641715990693
Precision:       0.3052668264735383
Recall:          0.27380631287216506
Exact hit@1:     0.305962156372724
Exact hit@5:     0.3802213495180293
Exact hit@any:   0.3827204569796501
Recall@20:       0.27380631287216506
MRR:             0.3382843712279628
Num predictions: 1.526597643698679


In [ ]:
#consecutive_correct_nodes, node_pred_from_text, ordered_pcst_nodes_20

In [36]:
ordered_pcst_nodes_20

{6517: array([ 17434, 107253,  14612,  52337,  64253,  42036, 101515, 101549,
        100748, 101536, 127330,  66826,  16510, 100741,  20710,  84409,
         23764, 105244, 113547,  22581]),
 9996: array([ 64835,  64338,  64521,  63426,  64520, 110565, 105370,  64778,
        107890, 110545,  64523,  79070,  88844, 100589,  64339, 103422,
         48463,  56752, 113018, 128709]),
 3630: array([14982, 14609, 15441, 15099, 15570, 17318, 15319, 15090, 15534,
        14893, 14137, 14979, 15430, 84853, 15416, 14245, 15454, 15369,
        14349, 15692]),
 10252: array([    43,   2140, 128049,   2447,  57057,  34410,   1641,   2050,
        128743, 128817,  12252, 123063, 128747,  54890,  56005,  62929,
          1671,  46401, 106257, 119960]),
 5142: array([38395, 30638, 38422, 32617, 24212, 29485, 30770, 38183, 23393,
        22759, 93542, 28347, 25704, 38445, 29037, 28240, 38184, 30670,
        28228, 84892]),
 1755: array([ 47408, 106871, 128128, 106670,  62592,  45556,  47288, 128127,
 

In [37]:
ordered_pcst_nodes_20_reindex = {i: v.tolist() for i, v in enumerate(ordered_pcst_nodes_20.values())}

In [38]:
ordered_pcst_nodes_20_reindex

{0: [17434,
  107253,
  14612,
  52337,
  64253,
  42036,
  101515,
  101549,
  100748,
  101536,
  127330,
  66826,
  16510,
  100741,
  20710,
  84409,
  23764,
  105244,
  113547,
  22581],
 1: [64835,
  64338,
  64521,
  63426,
  64520,
  110565,
  105370,
  64778,
  107890,
  110545,
  64523,
  79070,
  88844,
  100589,
  64339,
  103422,
  48463,
  56752,
  113018,
  128709],
 2: [14982,
  14609,
  15441,
  15099,
  15570,
  17318,
  15319,
  15090,
  15534,
  14893,
  14137,
  14979,
  15430,
  84853,
  15416,
  14245,
  15454,
  15369,
  14349,
  15692],
 3: [43,
  2140,
  128049,
  2447,
  57057,
  34410,
  1641,
  2050,
  128743,
  128817,
  12252,
  123063,
  128747,
  54890,
  56005,
  62929,
  1671,
  46401,
  106257,
  119960],
 4: [38395,
  30638,
  38422,
  32617,
  24212,
  29485,
  30770,
  38183,
  23393,
  22759,
  93542,
  28347,
  25704,
  38445,
  29037,
  28240,
  38184,
  30670,
  28228,
  84892],
 5: [47408,
  106871,
  128128,
  106670,
  62592,
  45556,
  47

In [39]:
merged_pred = {k: node_pred_from_text[k] + ordered_pcst_nodes_20_reindex[k] for k in node_pred_from_text}

In [41]:
ensemble_pred = {k: list(dict.fromkeys(node_pred_from_text[k] + ordered_pcst_nodes_20_reindex[k]))[:20] for k in node_pred_from_text}

In [43]:
compute_intermediate_metrics(consecutive_correct_nodes, ensemble_pred)

F1:              0.07871156793722726
Precision:       0.04687611567297393
Recall:          0.46843165500289
Exact hit@1:     0.3109603712959657
Exact hit@5:     0.48339878614780435
Exact hit@any:   0.5801499464476972
Recall@20:       0.46843165500289
MRR:             0.3848636999518817
Num predictions: 20.0


In [44]:
pcst_output['correct_nodes']

{6517: [56241, 55842, 56174, 56263],
 9996: [10833],
 3630: [15179],
 10252: [43, 2140],
 5142: [30638],
 1755: [62592],
 5686: [59232],
 7969: [52067, 53456, 46931, 40628, 103931],
 6641: [18223],
 10271: [14330, 14505, 14508, 14512, 14522, 14141],
 10069: [111],
 768: [99105, 99106, 99107, 33271, 99109, 30376, 31629, 99247, 94839, 27609],
 3093: [31754, 27547, 38252],
 9732: [45193, 47627, 50011, 103928, 112264],
 6708: [94418, 94210, 94316],
 5950: [27831],
 11077: [29113],
 4438: [8034,
  57603,
  61258,
  61259,
  61260,
  61261,
  8878,
  61262,
  61263,
  61264,
  61265,
  61266,
  61267,
  61268,
  61269,
  5874,
  57658,
  701],
 5928: [24225, 23564, 22757, 22759],
 8037: [63952, 64778, 63235, 64876],
 4259: [13664, 11205, 6887, 5448, 3145, 3147, 11641, 5884, 765],
 3815: [37440],
 5496: [96249, 96941],
 5772: [23162, 23915],
 10117: [108356, 44625, 106193, 103667, 46101, 42615, 111482, 48219, 112316],
 5699: [16019],
 7152: [94307, 22349, 22335],
 10551: [111],
 7224: [14558]